<a href="https://colab.research.google.com/github/rexian/ML/blob/main/langchain/vectorDB/document_embedding_with_pinecone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain langchain_community
!pip install pinecone-client
!pip install pypdf

In [ ]:
!pip install openai
!pip install tiktoken

In [5]:
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.vectorstores import Pinecone
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
import os

Extract the text from PDF files

In [6]:
loader = PyPDFDirectoryLoader("spaul")
data = loader.load()

Split the extracted text into chunks

In [8]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
text_chunks = text_splitter.split_documents(data)

Download and setup embedding

In [ ]:
import os
os.environ['OPENAI_API_KEY'] = ""
embeddings = OpenAIEmbeddings()

Initialize Pinecode

In [ ]:
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY', '')
PINECONE_API_ENV = os.environ.get('PINECONE_API_ENV', 'gcp-starter')

In [ ]:
import pinecone

# Initialize pinecone
pinecone.init(
    api_key=PINECONE_API_KEY,
    environment=PINECONE_API_ENV
)
index_name = "test" # put in the name of your pinecone index here

Create embedding for each chunk

In [ ]:
docsearch = Pinecone.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)

Creating an LLM wrapper

In [ ]:
llm = OpenAI()
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=docsearch.as_retriever())
query = 'What is Natural Language Processing?'
qa.run(query)